<a href="https://colab.research.google.com/github/anweshaban21/DeepSectoralAI/blob/main/BiLSTM_GARCH_BETA__SectoralAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 9.0 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2015-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-912965745.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [5]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [6]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [7]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [8]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-01,486.962891,436.521210,294.023834,624.794800
2015-01-02,497.853088,442.596436,302.369720,631.223572
2015-01-05,500.999146,438.859589,302.995636,628.691711
2015-01-06,483.090790,432.027802,290.143036,621.195801
2015-01-07,482.703583,433.288696,282.298004,631.198608
...,...,...,...,...
2025-08-18,1082.599976,2003.599976,1434.599976,2001.400024
2025-08-19,1083.199951,1991.099976,1436.300049,2029.900024
2025-08-20,1080.199951,1988.199951,1430.599976,2017.500000


In [9]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2015-01-02,0.022117,0.013821,0.027990,0.010237
2015-01-05,0.006299,-0.008479,0.002068,-0.004019
2015-01-06,-0.036400,-0.015690,-0.043344,-0.011995
2015-01-07,-0.000802,0.002914,-0.027411,0.015974
2015-01-08,0.006596,0.020788,0.026835,0.052957
...,...,...,...,...
2025-08-18,0.013391,0.006258,0.005101,0.011660
2025-08-19,0.000554,-0.006258,0.001184,0.014140
2025-08-20,-0.002773,-0.001458,-0.003976,-0.006127


In [10]:
pip install arch


In [11]:
from arch import arch_model
import pandas as pd
import numpy as np


In [12]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [13]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002847. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002803. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002725. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002817. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002462. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [ ]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2025-07-17  0.012921       0.015301  0.011752
2025-07-18  0.012463       0.014436  0.011646
2025-07-21  0.012098       0.013924  0.011590
2025-07-22  0.012811       0.013634  0.011693
2025-07-23  0.012370       0.013094  0.012026


LSTM

In [14]:
pip install numpy pandas scikit-learn tensorflow


In [15]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [27]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


def build_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model




In [28]:
def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [ ]:
bilstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting BiLSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    bilstm_volatility_data[sector] = sector_vols


Predicting BiLSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


...

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
print(bilstm_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2025-08-18    0.001192  0.001137  0.000711  0.000688
2025-08-19    0.001210  0.001088  0.000717  0.000646
2025-08-20    0.001239  0.000962  0.000732  0.000634
2025-08-21    0.001260  0.000986  0.000791  0.000665
2025-08-22    0.001275  0.001068  0.000820  0.000692


In [ ]:
bilstm_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2015-02-02         NaN       NaN       NaN       NaN
2015-02-03         NaN       NaN       NaN       NaN
2015-02-04         NaN       NaN       NaN       NaN
2015-02-05         NaN       NaN       NaN       NaN
2015-02-06         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2025-07-17    0.001556  0.000694  0.000763  0.000901
2025-07-18    0.001714  0.000666  0.000812  0.000888
2025-07-21    0.001742  0.000633  0.000852  0.000833
2025-07-22    0.001780  0.000654  0.000905  0.000806
2025-07-23    0.001819  0.000688  0.000939  0.000785

[2586 rows x 4 columns]>

In [ ]:
bilstm_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2015-02-02,NaN,NaN,NaN,NaN
2015-02-03,NaN,NaN,NaN,NaN
2015-02-04,NaN,NaN,NaN,NaN
2015-02-05,NaN,NaN,NaN,NaN
2015-02-06,NaN,NaN,NaN,NaN


In [31]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [32]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [33]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [34]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.729562  0.817045  0.670946  0.722533     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.358161     0.993063   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000    0.000000      0.000000   
Banking      1.337353      1.012297  0.000000    0.000000      0.000000   
Pharma       0.000000      0.000000  0.529173    0.506275      0.677415   
Auto         0.000000      0.000000  0.000000    0.000000      0.000000   
FMCG         0.000000      0.000000  0.000000    0.000000      0.000000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS    IT

Computing Sharpe Ratio

In [35]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [36]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, bilstm_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'BiLSTM_Vol', 'Sharpe'])


In [37]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'BiLSTM_Vol', 'Sharpe'])


In [38]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [ 0.01794877 -3.82121726  3.13517715]
Intercept: 0.07014960631634984


In [39]:
features_df['Risk_Score'] = X @ weights


In [40]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
0  HCLTECH.NS   -0.045627
3    WIPRO.NS   -0.044108
1     INFY.NS   -0.043294

Top 3 safest stocks in Banking:
          Stock  Risk_Score
4   AXISBANK.NS   -0.046922
6  ICICIBANK.NS   -0.041733
7  KOTAKBANK.NS   -0.039327

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS   -0.051135
9     DRREDDY.NS   -0.049543
8       CIPLA.NS   -0.049312

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.063945
11         M&M.NS   -0.047006
12      MARUTI.NS   -0.038953

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
14       DABUR.NS   -0.041474
16         ITC.NS   -0.041173
15  HINDUNILVR.NS   -0.038887


BiLSTM AND GARCH ONLY

In [41]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'BiLSTM_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, LSTM):", weights_new)


New Weights (GARCH, LSTM): [-2.34870516  1.37515055]


In [44]:
features_df['Risk_Score_BiLSTM_GARCH'] = X_new @ weights_new


In [45]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + BiLSTM only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_BiLSTM_GARCH')
          .head(3)[['Stock', 'Risk_Score_BiLSTM_GARCH']])



Safest stocks in IT (GARCH + BiLSTM only):
        Stock  Risk_Score_BiLSTM_GARCH
0  HCLTECH.NS                -0.036853
1     INFY.NS                -0.036350
3    WIPRO.NS                -0.035594

Safest stocks in Banking (GARCH + BiLSTM only):
          Stock  Risk_Score_BiLSTM_GARCH
4   AXISBANK.NS                -0.044588
6  ICICIBANK.NS                -0.041151
7  KOTAKBANK.NS                -0.036011

Safest stocks in Pharma (GARCH + BiLSTM only):
           Stock  Risk_Score_BiLSTM_GARCH
10  SUNPHARMA.NS                -0.039750
9     DRREDDY.NS                -0.036830
8       CIPLA.NS                -0.036651

Safest stocks in Auto (GARCH + BiLSTM only):
            Stock  Risk_Score_BiLSTM_GARCH
13  TATAMOTORS.NS                -0.056136
11         M&M.NS                -0.041470
12      MARUTI.NS                -0.035948

Safest stocks in FMCG (GARCH + BiLSTM only):
            Stock  Risk_Score_BiLSTM_GARCH
16         ITC.NS                -0.034080
14       DABUR.NS    

BiLSTM AND BETA ONLY

In [46]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'BiLSTM_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", LSTM =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = 0.00440893990773445 , LSTM = -21.614585822526543
Intercept: 0.049159661555448805


In [47]:
features_df['Risk_Score_BiLSTM_Beta'] = X_lb @ weights_lb


In [49]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (BiLSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_BiLSTM_Beta')
          .head(3)[['Stock', 'Risk_Score_BiLSTM_Beta']])



Safest stocks in IT (BiLSTM + Beta only):
        Stock  Risk_Score_BiLSTM_Beta
0  HCLTECH.NS               -0.026563
1     INFY.NS               -0.024815
2      TCS.NS               -0.022115

Safest stocks in Banking (BiLSTM + Beta only):
          Stock  Risk_Score_BiLSTM_Beta
4   AXISBANK.NS               -0.023925
6  ICICIBANK.NS               -0.023306
7  KOTAKBANK.NS               -0.021818

Safest stocks in Pharma (BiLSTM + Beta only):
           Stock  Risk_Score_BiLSTM_Beta
10  SUNPHARMA.NS               -0.030196
9     DRREDDY.NS               -0.028835
8       CIPLA.NS               -0.017411

Safest stocks in Auto (BiLSTM + Beta only):
            Stock  Risk_Score_BiLSTM_Beta
13  TATAMOTORS.NS               -0.033206
12      MARUTI.NS               -0.027733
11         M&M.NS               -0.022007

Safest stocks in FMCG (BiLSTM + Beta only):
            Stock  Risk_Score_BiLSTM_Beta
16         ITC.NS               -0.021521
14       DABUR.NS               -0.020060
15

GARCH AND BETA ONLY

In [50]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = 0.01779233166850252 , GARCH = -3.5819806916440062
Intercept: 0.0702174193720433


In [51]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [52]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
        Stock  Risk_Score_GARCH_Beta
0  HCLTECH.NS              -0.046114
1     INFY.NS              -0.043657
3    WIPRO.NS              -0.043374

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
4   AXISBANK.NS              -0.046738
6  ICICIBANK.NS              -0.041798
7  KOTAKBANK.NS              -0.039459

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
10  SUNPHARMA.NS              -0.051790
9     DRREDDY.NS              -0.050176
8       CIPLA.NS              -0.048397

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
13  TATAMOTORS.NS              -0.063929
11         M&M.NS              -0.046659
12      MARUTI.NS              -0.039912

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
14       DABUR.NS              -0.041393
16         ITC.NS              -0.041250
15  HINDUNILVR.NS         

In [53]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_BiLSTM_GARCH,Risk_Score_BiLSTM_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,-0.045627,-0.036853,-0.026563,-0.046114
1,INFY.NS,-0.043294,-0.036350,-0.024815,-0.043657
2,TCS.NS,-0.039351,-0.032229,-0.022115,-0.039647
3,WIPRO.NS,-0.044108,-0.035594,-0.016865,-0.043374
4,AXISBANK.NS,-0.046922,-0.044588,-0.023925,-0.046738


Comparison per sector

In [54]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [55]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
0,HCLTECH.NS,-0.045627
3,WIPRO.NS,-0.044108
1,INFY.NS,-0.043294



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_GARCH:


,Stock,Risk_Score_BiLSTM_GARCH
0,HCLTECH.NS,-0.036853
1,INFY.NS,-0.036350
3,WIPRO.NS,-0.035594



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_Beta:


...

,Stock,Risk_Score_BiLSTM_GARCH
10,SUNPHARMA.NS,-0.039750
9,DRREDDY.NS,-0.036830
8,CIPLA.NS,-0.036651



🔹 Top 3 safest stocks using Risk_Score_BiLSTM_Beta:


,Stock,Risk_Score_BiLSTM_Beta
10,SUNPHARMA.NS,-0.030196
9,DRREDDY.NS,-0.028835
8,CIPLA.NS,-0.017411



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
10,SUNPHARMA.NS,-0.051790
9,DRREDDY.NS,-0.050176
8,CIPLA.NS,-0.048397


Comparison table

In [56]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_BiLSTM_GARCH', 'Risk_Score_BiLSTM_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [57]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_BiLSTM_GARCH_Rank,Risk_Score_BiLSTM_Beta_Rank,Risk_Score_GARCH_Beta_Rank
14,DABUR.NS,1.0,2.0,2.0,1.0
16,ITC.NS,2.0,1.0,1.0,2.0
15,HINDUNILVR.NS,3.0,3.0,3.0,3.0
